In [1]:
import numpy as np

In [2]:
import numpy as np

# Define the domain
x = np.linspace(0, 2, 1000)  # 1000 points between 0 and 2

# Define the function to be integrated
y = x  # since we are integrating x

# Perform the integration using the trapezoidal rule
integral = np.trapz(y, x)

print(f"The integral of x from 0 to 2 is approximately {integral}")


The integral of x from 0 to 2 is approximately 2.0


In [5]:
import numpy as np

# Define the function to be integrated
def f(x, y):
    return x * y

# Define the domain for x and y
x = np.linspace(0, 2, 100)
y = np.linspace(0, 3, 100)

# Create a grid of points
X, Y = np.meshgrid(x, y)

# Evaluate the function on the grid
Z = f(X, Y)

# Perform the double integration using the trapezoidal rule
# First, integrate with respect to y for each fixed x
integral_y = np.trapz(Z, y, axis=0)

# Then, integrate the result with respect to x
integral = np.trapz(integral_y, x)

print(f"The double integral of xy over [0, 2] x [0, 3] is approximately {integral}")


The double integral of xy over [0, 2] x [0, 3] is approximately 9.0


In [ ]:
def GF_table(x, casex: str, casey: str, L):
    # returns the eigenfunc(x) Xm, the eigenvalue betam and the norm Nx

    m = np.arange(1, len(x))
    if (casex == 'D' and casey == 'D'): #11
        beta_m = m*np.pi/L
        X_m = np.sin(beta_m*x)
        Nx = L/2
        return X_m, beta_m, Nx
    
    if (casex == 'D' and casey == 'N'): #12
        beta_m = (2*m-1)*np.pi/(2*L)
        X_m = np.sin(beta_m*x)
        Nx = L/2
        return
    if (casex == 'D' and casey == 'D'): #13
        return
    

def green_function(x, y, xs, ys, casex: str, casey: str):



In [14]:
x = np.arange(1, 20, 2)
L=10
x

array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19])

In [15]:
m = np.arange(1, len(x)+1)
m

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [16]:
beta = m*np.pi/L
beta

array([0.31415927, 0.62831853, 0.9424778 , 1.25663706, 1.57079633,
       1.88495559, 2.19911486, 2.51327412, 2.82743339, 3.14159265])

In [18]:
X = np.sin(beta*x)
X

array([ 3.09016994e-01,  9.51056516e-01, -1.00000000e+00,  5.87785252e-01,
        1.00000000e+00,  9.51056516e-01, -3.09016994e-01, -1.46957616e-15,
       -8.09016994e-01,  5.87954260e-15])